In [ ]:
import serial
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import pandas as pd
import serial

In [ ]:
def generate_digital_signal_n(fo, ts, num_samples=1024):
    # Create an array to represent the sample number
    n = np.arange(0, num_samples, 1)
    digital_signal_n = np.sin(2 * np.pi * fo * n * ts)
    return n, digital_signal_n

def generate_digital_signal_t(fo, ts, num_samples=1024):
    ts_32 = np.float32(ts)
    # Create an array to represent time
    t = np.arange(0, num_samples * ts_32, ts_32)
    digital_signal_t = np.sin(2 * np.pi * fo * t)
    return t, digital_signal_t

In [ ]:
def voltage_to_adc(voltage_signal, resolution, adc_range=5.0):

   # Convert voltage to ADC samples
   adc_samples = np.round((voltage_signal / adc_range) * (2**resolution - 1))

   return adc_samples.astype(int)

In [ ]:
def plot_adc_freqs(fo, fs, batch_size=1024, resolution=12, adc_range=5.0, scale=1.0, xlim=None, ylim=None):
   # Sampling time period
   ts = 1 / fs
   # create an array to represent the sample number
   n = np.arange(0, batch_size*scale, 1)
   digital_signal_n = 2.5+np.sin(2 * np.pi * fo * n * ts)

   ts_32 = np.float32(ts)  # Convert ts to a float32 = ts.astype(np.float32)  # Convert ts to a float32
   t = np.arange(0, batch_size*scale* ts_32, ts_32)  # Use astype instead of asType
   digital_signal_t = 2.5+np.sin(2 * np.pi * fo * t)  # Replace this with your actual digital signal

   # create adc signals
   adc_signal_n = voltage_to_adc(digital_signal_n, resolution, adc_range)
   adc_signal_t = voltage_to_adc(digital_signal_t, resolution, adc_range)

   # Create subplots
   plt.figure(figsize=(15, 10))
   # Plot the first subplot
   plt.subplot(2, 1, 1)
   plt.stem(n, adc_signal_n)
   plt.title("Theoretical Waveform - Sample Number")
   plt.xlabel("Sample Number")
   plt.ylabel("Amplitude")
   plt.grid(True)
   plt.xlim(0, xlim)

   # Plot the second subplot

   plt.subplot(2, 1, 2)
   plt.stem(t, adc_signal_t)
   plt.title("Theoretical Waveform - Time")
   plt.xlabel("Time")
   plt.ylabel("Amplitude")
   plt.grid(True)
   plt.xlim(0, xlim*ts)


   plt.tight_layout()
   plt.show()

In [ ]:

def plot_adc_signal(samples, excel_file, fs=208000, resolution=12, adc_range=5.0, xlim=1024, ylim=None, length=1024):
    # Calculate the time period
    ts = 1 / fs

    # Ensure length does not exceed the array size
    length = min(length, len(samples))

    # Create an array to represent the sample number
    n = np.arange(0, length, 1)
    voltage_signal_n = samples[0:length]  # Assuming the input array is the

    # Create an array to represent time
    ts_32 = np.float32(ts)
    t = np.arange(0, length * ts_32, ts_32)
    voltage_signal_t = samples[0:length]  # Assuming the input array is the

    # Create subplots
    plt.figure(figsize=(20, 5))

    # Plot the ADC signal against sample number
    plt.subplot(2, 1, 1)
    plt.stem(n, voltage_signal_n)
    plt.title("ADC Signal - Sample Number")
    plt.xlabel("Sample Number")
    plt.ylabel("ADC Sample")
    plt.grid(True)
    plt.xlim(0, xlim)

    # Plot the ADC signal against time
    plt.subplot(2, 1, 2)
    plt.stem(t, voltage_signal_t)
    plt.title("ADC Signal - Time")
    plt.xlabel("Time")
    plt.ylabel("ADC Sample")
    plt.grid(True)
    plt.xlim(0, xlim / fs)

    plt.tight_layout()
    plt.show()

    # Append the values to an Excel sheet
    df = pd.DataFrame({
        'Sample Number': n,
        'ADC Sample': voltage_signal_n,
        'Time': t,
        'ADC Sample (Time)': voltage_signal_t
        })

    try:
        # Load existing data from the Excel file
        existing_data = pd.read_excel(excel_file)
        df = pd.concat([existing_data, df], ignore_index=True)
    except FileNotFoundError:
        pass  # File does not exist, ignore and create a new one

    # Write the updated DataFrame to the Excel file
    df.to_excel(excel_file, index=False)

    


In [ ]:
# Plot the FFT of the ADC signals
def plot_adc_signal_fft_magnitudes(FFTsamples1, fs=208000, resolution=12, adc_range=5.0, xlim=1024, ylim=None):
   # freq1 = np.arange(0, fs / 2, fs / 1024)
   n = np.arange(0, 512, 1)

   
   plt.figure(figsize=(15, 5))
   plt.stem(n, FFTsamples1)
   plt.title("FFT of ADC Signal 1")
   plt.xlabel("Frequency INDEX")
   plt.ylabel("Amplitude")
   plt.xlim(0, len(FFTsamples1))
   plt.ylim(-40, 20)
   plt.grid(True)
   plt.tight_layout()
   plt.show()
   plt.close()

In [ ]:
def plot_adc_signal_fft_dBV(dBV_fft_samples, fft_freqs, fs=208000, resolution=12, adc_range=5.0, xlim=1024, ylim=None):
   plt.clf()

   plt.figure(figsize=(20, 3))

   plt.stem(fft_freqs, dBV_fft_samples, markerfmt='.', basefmt='b-', linefmt='b-')

   plt.title("FFT of ADC Signal 1")
   plt.xlabel("Frequency Hz")
   plt.ylabel("dBV")

   # Set x-axis to be logarithmic
   #plt.xscale('log')

   plt.xlim(0, fs/2)
   #plt.ylim(-40, 30)
   plt.grid(True)
   plt.show()
   plt.close()

In [ ]:
# read the data from the Excel file
df = pd.read_excel('ArduinoSampleSignal1.xlsx')

# n samples, ch0 samples, ch1 samples
n = df['n']
ch0 = df['ch0 samples']

# plot the fft of the signal
# plot_adc_signal_fft_magnitudes(ch0[0:512], xlim=512, ylim=None)


In [9]:
# Specify the COM port and baud rate
comPort = "COM9"
baudRate = 115200

# Create a new SerialPort object
serialPort = serial.Serial(comPort, baudRate)
fs = 208000
xlim = 1024
ch0_data = []
batch_size = 1024
large_ch0_data_values = []
count = 0
fft_freqs = []
freqready = 0
try:
    while True:
        data = serialPort.readline().decode('utf-8').strip()

        if data.startswith("CH0:"):
            # Process CH0 data
            ch0_data_values = [float(value) for value in [serialPort.readline().decode('utf-8').strip() for _ in range(batch_size)]]

            # Plot the ADC signal
            #plot_adc_signal(ch0_data_values,"longsingalvalues.xlsx", fs=fs, resolution=12, adc_range=5.0, xlim=xlim/4, ylim=None, length=batch_size)
            print(ch0_data_values)
        elif data.startswith("CH0 FFT:"):
            # Process CH0 FFT data
            ch0_fft_data_values = [float(value) for value in [serialPort.readline().decode('utf-8').strip() for _ in range(batch_size//2)]]

            # Plot the FFT of the ADC signal
            #plot_adc_signal_fft_magnitudes(ch0_fft_data_values, fs=fs, resolution=12, adc_range=5.0, xlim=256, ylim=None)
            # plot_adc_signal_fft_dBV(ch0_fft_data_values, fs=fs, resolution=12, adc_range=5.0, xlim=256, ylim=None)
            if freqready == 1:
                plot_adc_signal_fft_dBV(ch0_fft_data_values, fft_freqs, fs=fs, resolution=12, adc_range=5.0, xlim=fs/2, ylim=None)
                freqready = 0
        elif data.startswith("CH0 Freqs:"):
            # Process CH0 FFT data
            fft_freqs = [float(value) for value in [serialPort.readline().decode('utf-8').strip() for _ in range(batch_size//2)]]
            freqready = 1
        else:
            # Handle other data or ignore if needed
            pass

except serial.SerialException as e:
    print(f"Error: {e}")
except KeyboardInterrupt:
    print("Serial reading terminated by the user.")
finally:
    serialPort.close()